In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from scipy.signal import savgol_filter

# Load data
filename = '../npz/BTCUSD_1T.npz'
data = np.load(filename, allow_pickle=True)['data']
loaded_df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

# Convert 'timestamp' to datetime and set it as the index
loaded_df['timestamp'] = pd.to_datetime(loaded_df['timestamp'])
loaded_df.set_index('timestamp', inplace=True)

# Convert all price and volume columns to numeric format
for column in ['open', 'high', 'low', 'close', 'volume']:
    loaded_df[column] = pd.to_numeric(loaded_df[column], errors='coerce')

# Create interactive widgets
n_slider = widgets.IntSlider(min=1000, max=500 * 1000, step=1000, value=100000, description='n', layout=widgets.Layout(width='600px'))
w1_slider = widgets.IntSlider(min=100, max=20000, step=100, value=5000, description='w1', layout=widgets.Layout(width='600px'))
w2_slider = widgets.IntSlider(min=100, max=40000, step=100, value=10000, description='w2', layout=widgets.Layout(width='600px'))
window_slider = widgets.IntSlider(min=5, max=40000, step=1, value=6000, description='savgol window', layout=widgets.Layout(width='600px'))
# Function to update the plot
def update_plot2(n, w1, w2, savgol_window):
    # Select the last n rows
    df = loaded_df.tail(n)
    closes = df['close']
    polyorder = 3       # Степень полинома
    delta = 1           # Временной шаг между точками (замените на реальное значение, если нужно)
    w1_fixed = w1 / 2 == 0 and w1 + 1 or w1
    w1_smoothed = savgol_filter(closes, window_length=w1_fixed,
                            polyorder=polyorder, deriv=0, delta=delta)
    w2_fixed = w2 / 2 == 0 and w2 + 1 or w2
    w2_smoothed = savgol_filter(closes, window_length=w2_fixed,
                            polyorder=polyorder, deriv=0, delta=delta)
    diff = (w2_smoothed - w1_smoothed) * (w2_smoothed[0]/w2_smoothed)
    ''' давай попробуем такую простую идею. цену биткоина можно представить как сумму значений скользящей средней и 
    текущей цены. пусть цена p, а скользящая средняя ma. возьмем разницу между ними d = p - ma.
    тогда мы можем сделать и обратную операцию p = ma + d.
    продолжим рассуждать. можно взять скользящие разных периодов. например, ma10 и ma30 - скользящие по 10 и 30 точкам.
    для определенности под ценой будем считать цену закрытия минутных баров. соотвественно скользящие будут за 10 и 30 минут.
    посмотрим
    давай поговорим на тему среднесрочных торговых стратегий. если взять разность скользящих средних, то получится график
    напоминающий хаотические колебания. 
    
    '''
    
    # Set window_length and polyorder according to your data characteristics.
    # window_length должно быть нечетным и больше polyorder.
    window_length = savgol_window / 2 == 0 and savgol_window + 1 or savgol_window

    # сглаживаем
    smoothed = savgol_filter(diff, window_length=window_length,
                            polyorder=polyorder, deriv=0, delta=delta)

    # Вычисляем первую производную
    deriv_sg = savgol_filter(diff, window_length=window_length,
                            polyorder=polyorder, deriv=1, delta=delta)
    # smoothen it once again
    deriv_sg = savgol_filter(deriv_sg, window_length=window_length,
                            polyorder=polyorder, deriv=0, delta=delta)

    ma_w1 = closes.rolling(window=int(w1/4)).mean().bfill()
    ma_w2 = closes.rolling(window=int(w2/4)).mean().bfill()
    ma_diff = (ma_w2 - ma_w1) * (ma_w2.iloc[-1]/ma_w2)

    plt.figure(figsize=(15, 3))
    plt.plot(closes.index, closes, label='close')
    plt.plot(closes.index, w1_smoothed, label='w1')
    plt.plot(closes.index, w2_smoothed, label='w2')
    plt.plot(closes.index, ma_w1, label='ma_w1')
    plt.plot(closes.index, ma_w2, label='ma_w2')
    plt.legend()

    plt.figure(figsize=(15, 3))
    smoothed_series = pd.Series(smoothed, index=closes.index)

    # ax = diff.plot(label='diff')
    # # Explicitly plot smoothed_series on the same axes using its index and values
    # ax.plot(smoothed_series.index, smoothed_series.values, label='smoothed')
    #print(smoothed_series)
    
    plt.plot(closes.index, diff, label=f'diff')
    plt.plot(smoothed_series, label='smoothed')
    plt.plot(closes.index, ma_diff, label='ma diff')
    plt.legend()

    # plt.figure(figsize=(15, 3))
    # plt.plot(closes.index, deriv_sg, label='deriv_sg')
    # plt.legend()
    
    # plt.figure(figsize=(3, 3))
    # plt.plot(smoothed_series, deriv_sg, label='fp')
    # plt.legend()

    # plt.figure(figsize=(15, 5))
    # # plot the same but without dates on X axis
    # plt.plot(diff.values, label='diff')
    # plt.plot(smoothed, label='smoothed')
    # plt.legend()
    # plt.figure(figsize=(15, 5))
    # plt.plot(smoothed, label='smoothed')
    # plt.legend()
    # plt.figure(figsize=(15, 5))
    # plt.plot(diff.values - smoothed, label='diff - smoothed')
    # plt.legend()
    
    plt.show()

# Use interact to create the interactive plot
interact(update_plot2, n=n_slider, w1=w1_slider, w2=w2_slider, savgol_window=window_slider)

In [ ]:
series = [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116]
N = len(series)
tau = 2  # Лаг
m = 3

In [ ]:
for i in range(m):
    print(series[i:N - (m-1) * tau : tau])
